<a href="https://colab.research.google.com/github/AlbertoRoldanSastre/AlbertoRoldanSastre/blob/main/datos_YOLO_hasta_dataaug_31_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PASOS PARA LA DESCARGA DEL DATASET Y PREPARACIÓN

Crear el ambiente virtual para que incluso si muevo archivos funcione:

En esta hoja voy a detallar todos los procesos que me han funcionado para poder descargar el dataset, transformar  y dejar todo listo para correr el modelo:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# 0) Conectar con Drive

In [1]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


# 1) Descargar el dataset con las anotaciones desde Roboflow

cuando descargo versión, lo hago produciendo versión con auto-oriented, nada más, no tiling no resize (esto la haré luego yo)

In [ ]:
#if not installed, you need to install roboflow in terminal  : pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="9eeIO2PsJSso62o9")

project = rf.workspace("seals-cqpqc").project("aws-qgxmp")          # nombre del workspace y del projecto
dataset = project.version(10).download("YOLOV8")                     # "6" ES la versión que descargué , adaptala a la tuya. YOLOV8 es el formato


#2) Transformar las anotaciones a formato YOLO (es decir, bounding box)

esto es porque al decargar anotaciones, yo tengo poligonos y bounding boxes, quiero que todas sean bounding boxed que es lo que el modelo acepta

Haces carpeta a carpeta (train- validation-test)

otra opción es: Use a framework that accepts mixed annotations without modification (e.g., PyTorch with COCO JSON format).

In [ ]:
import os

def polygon_to_bbox(coords):
    """
    Convierte una lista de coordenadas de un polígono (x1, y1, x2, y2, ..., xN, yN)
    a una bounding box en formato YOLO: (x_center, y_center, width, height),
    asumiendo que las coordenadas están normalizadas [0,1].
    """
    xs = coords[0::2]  # extrae todos los valores de x
    ys = coords[1::2]  # extrae todos los valores de y

    x_min = min(xs)
    x_max = max(xs)
    y_min = min(ys)
    y_max = max(ys)

    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    width = x_max - x_min
    height = y_max - y_min

    return x_center, y_center, width, height

def process_label_file(src_file, dst_file):
    """
    Lee un archivo de anotaciones en formato YOLO (líneas con la clase seguida de coordenadas).
    Si la línea contiene 4 números (bounding box), la deja igual.
    Si contiene más de 4 (asumiendo un polígono), lo convierte a bounding box.
    Escribe el resultado en un nuevo archivo.
    """
    with open(src_file, 'r') as f_in:
        lines = f_in.readlines()

    new_lines = []
    for line in lines:
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        cls = parts[0]  # la clase (por ejemplo, "0", "1", "2", etc.)       ##clases indicadas en data.yaml
        coords = list(map(float, parts[1:]))

        if len(coords) == 4:
            # Ya es bounding box, se deja igual.
            new_line = f"{cls} {coords[0]:.16f} {coords[1]:.16f} {coords[2]:.16f} {coords[3]:.16f}"
        else:
            # Es un polígono; se convierte a bounding box.
            x_center, y_center, width, height = polygon_to_bbox(coords)
            new_line = f"{cls} {x_center:.16f} {y_center:.16f} {width:.16f} {height:.16f}"      #puedes modificar el número de decimales(16, o los que quieras)
        new_lines.append(new_line)

    with open(dst_file, 'w') as f_out:
        for new_line in new_lines:
            f_out.write(new_line + "\n")

    print(f"Procesado: {os.path.basename(src_file)} -> {os.path.basename(dst_file)}")

def process_all_labels(src_labels_dir, dst_labels_dir):
    """
    Procesa todos los archivos .txt en src_labels_dir y guarda los nuevos archivos
    (con anotaciones convertidas a bounding boxes) en dst_labels_dir.
    """
    os.makedirs(dst_labels_dir, exist_ok=True)
    for filename in os.listdir(src_labels_dir):
        if filename.endswith(".txt"):
            src_path = os.path.join(src_labels_dir, filename)
            dst_path = os.path.join(dst_labels_dir, filename)
            process_label_file(src_path, dst_path)

if __name__ == "__main__":
    ###############################################################################################33
    # Rutas de entrada y salida: modifica estas rutas a las de tu sistema
    src_labels_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/valid/labels"
    dst_labels_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/valid/bblabels"

    ################################################################################################
    process_all_labels(src_labels_dir, dst_labels_dir)

    print("¡Conversión completada!")

#3) Comprobar que esas anotaciones que ha hecho, corresponden con las imágenes y están bien hechas

SOLO UNA IMAGEN

Saldrá la imagen con las anotaciones.

In [ ]:
!pip install opencv-python

In [ ]:
#import supervision, cv2 and numpy si lo necesitas en terminal, como antes: pip install supervision, cv2, numpy
!pip install supervision
!pip install cv2
!pip install numpy
import supervision as sv
import cv2
import numpy as np

# Load the image
image = cv2.imread("/content/drive/MyDrive/DSC00136_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.5d77c8e89e71cb168723b4a6ee832250_tile_91.jpg")

# Load the labels from the txt file
with open("/content/drive/MyDrive/DSC00136_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.5d77c8e89e71cb168723b4a6ee832250_tile_91.txt", "r") as f:
    lines = f.readlines()

# Parse the labels
xyxy = []
confidence = []
class_id = []
for line in lines:
    class_id_, x_center, y_center, width, height = map(float, line.strip().split())
    x1 = (x_center - width/2) * image.shape[1]
    y1 = (y_center - height/2) * image.shape[0]
    x2 = (x_center + width/2) * image.shape[1]
    y2 = (y_center + height/2) * image.shape[0]
    xyxy.append([x1, y1, x2, y2])
    confidence.append(1.0)  # Assuming confidence of 1.0 for all detections
    class_id.append(class_id_)

# Convert to supervision Detections object
if len(xyxy) == 0:
    print("No hay detecciones, se omite la construcción de sv.Detections")
else:
    detections = sv.Detections(
        xyxy=np.array(xyxy),
        confidence=np.array(confidence),
        class_id=np.array(class_id).astype(int)
    )

# Create annotators
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Annotate the image
annotated_frame = box_annotator.annotate(scene=image.copy(), detections=detections)
annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=[f"Class {int(class_id)}" for class_id in detections.class_id])

# Display the annotated image
sv.plot_image(image=annotated_frame, size=(16, 16))

#4. Tiling en carpetas grandes


**    IMPORTANTE     **

Aquí he de cambiar el tamaño de tiles que yo quierO

4.1 Generando todo en una carpeta    
(también está el código para hacer subcarpetas de cada imagen con tiles y con txt)
para cada imagen , partiendo de imagen y txt, crea tiles que van asociadas a txt, por lo tanto defines rutas entrada, salida y tamaño tiles

In [14]:
import cv2
import os
import numpy as np

def load_annotations(label_path, image_width, image_height):
    """
    Lee el archivo de anotaciones en formato YOLO (bounding boxes o polígonos)
    y devuelve una lista de anotaciones en coordenadas absolutas (en píxeles):
    (cls, x1, y1, x2, y2)
    """
    annotations = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls = parts[0]
            coords = list(map(float, parts[1:]))

            if len(coords) == 4:
                # Ya es bounding box (normalizada: x_center, y_center, w, h)
                x_center, y_center, w, h = coords
                abs_x_center = x_center * image_width
                abs_y_center = y_center * image_height
                abs_w = w * image_width
                abs_h = h * image_height
                x1 = abs_x_center - abs_w/2
                y1 = abs_y_center - abs_h/2
                x2 = abs_x_center + abs_w/2
                y2 = abs_y_center + abs_h/2
            else:
                # Es un polígono: convertir a bounding box envolvente
                xs = coords[0::2]
                ys = coords[1::2]
                x1 = min(xs) * image_width
                x2 = max(xs) * image_width
                y1 = min(ys) * image_height
                y2 = max(ys) * image_height

            annotations.append((cls, x1, y1, x2, y2))
    return annotations

def adjust_annotation_for_tile(annotation, tile_x, tile_y, tile_size):
    """
    Dada una anotación en coordenadas absolutas (x1, y1, x2, y2) de la imagen original,
    y el offset (tile_x, tile_y) del tile, calcula la intersección entre la caja y el tile.
    Devuelve la bounding box transformada a coordenadas locales del tile (normalizadas)
    o None si no hay intersección.
    """
    cls, x1, y1, x2, y2 = annotation
    tile_x1, tile_y1 = tile_x, tile_y
    tile_x2, tile_y2 = tile_x + tile_size, tile_y + tile_size

    # Calcular la intersección
    inter_x1 = max(x1, tile_x1)
    inter_y1 = max(y1, tile_y1)
    inter_x2 = min(x2, tile_x2)
    inter_y2 = min(y2, tile_y2)

    if inter_x2 <= inter_x1 or inter_y2 <= inter_y1:
        return None  # No hay intersección

    # Convertir a coordenadas locales del tile
    local_x1 = inter_x1 - tile_x
    local_y1 = inter_y1 - tile_y
    local_x2 = inter_x2 - tile_x
    local_y2 = inter_y2 - tile_y

    box_w = local_x2 - local_x1
    box_h = local_y2 - local_y1
    x_center = local_x1 + box_w / 2.0
    y_center = local_y1 + box_h / 2.0
    norm_x_center = x_center / tile_size
    norm_y_center = y_center / tile_size
    norm_w = box_w / tile_size
    norm_h = box_h / tile_size

    return (cls, norm_x_center, norm_y_center, norm_w, norm_h)

def process_image_tiling(image_path, label_path, tile_size, output_images_dir, output_labels_dir, overlap=0):
    """
    Realiza el tiling de una imagen y ajusta sus anotaciones.
    Guarda cada tile en output_images_dir y su etiqueta correspondiente en output_labels_dir.
    'overlap' (en píxeles) permite solapar los tiles.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error al cargar {image_path}")
        return
    orig_h, orig_w, _ = image.shape

    annotations = load_annotations(label_path, orig_w, orig_h)

    tile_idx = 0
    image_tile_count = 0
    label_tile_count = 0
    step = tile_size - overlap
    for y in range(0, orig_h, step):
        for x in range(0, orig_w, step):
            tile = image[y:min(y+tile_size, orig_h), x:min(x+tile_size, orig_w)]
            tile_h, tile_w, _ = tile.shape
            if tile_h < tile_size or tile_w < tile_size:
                padded_tile = np.zeros((tile_size, tile_size, 3), dtype=tile.dtype)
                padded_tile[0:tile_h, 0:tile_w] = tile
                tile = padded_tile

            base_name = os.path.splitext(os.path.basename(image_path))[0]
            tile_filename = f"{base_name}_tile_{tile_idx}.jpg"
            cv2.imwrite(os.path.join(output_images_dir, tile_filename), tile)
            image_tile_count += 1

            tile_annotations = []
            for ann in annotations:
                adj_ann = adjust_annotation_for_tile(ann, x, y, tile_size)
                if adj_ann is not None:
                    tile_annotations.append(adj_ann)

            label_filename = f"{base_name}_tile_{tile_idx}.txt"
            with open(os.path.join(output_labels_dir, label_filename), "w") as f:
                for ann in tile_annotations:
                    cls, xc, yc, w, h = ann
                    f.write(f"{cls} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")
            label_tile_count += 1

            tile_idx += 1

    print(f"Procesada {os.path.basename(image_path)}: {tile_idx} tiles generados.")
    print(f"En total, {image_tile_count} tiles de imágenes generados.")
    print(f"En total, {label_tile_count} tiles de etiquetas generados.")

    return image_tile_count, label_tile_count

def process_dataset(source_images_dir, source_labels_dir, output_images_dir, output_labels_dir, tile_size=960, overlap=0):
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)

    image_files = [f for f in os.listdir(source_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    total_image_tiles = 0
    total_label_tiles = 0

    for img_file in image_files:
        image_path = os.path.join(source_images_dir, img_file)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        label_path = os.path.join(source_labels_dir, label_file)
        if not os.path.exists(label_path):
            print(f"Advertencia: No se encontró etiqueta para {img_file}. Saltando...")
            continue

        image_tiles, label_tiles = process_image_tiling(image_path, label_path, tile_size, output_images_dir, output_labels_dir, overlap)
        total_image_tiles += image_tiles
        total_label_tiles += label_tiles

    # Mostrar los resultados finales
    print(f"\nTotal de tiles generados en la carpeta de imágenes: {total_image_tiles}")
    print(f"Total de tiles generados en la carpeta de etiquetas: {total_label_tiles}")
    print("Tiling de todas las imágenes completado.")

if __name__ == "__main__":
    source_images_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/train/images"
    source_labels_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/train/bblabels"
    output_images_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/train/tiles/images"
    output_labels_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/train/tiles/labels"

    tile_size = 640
    overlap = 128
     # Puedes ajustar o poner 0 si no quieres solapamiento. Puedes ajustar o poner 0 si no quieres solapamiento.  ## mejor solapamiento?  --> chatgp:  -->Sí, solapar (overlap) los tiles es una práctica común. Cuando un objeto (o una caja) cae justo en el borde de un tile, si no hay solapamiento existe el riesgo de que la caja se corte y se pierda parte de la información. Con un solapamiento, ese objeto puede aparecer (completamente o en mayor medida) en el tile adyacente. Esto mejora la probabilidad de que el objeto se detecte correctamente.

                  #debes poner un overlap del 15-20% minimo, para no perder objetos
                  #cómo calcular?:
                  #tamaño tile x (%), digamos 15 (es decir, 0,15)= 96 pixeles de overlap debo poner

    process_dataset(source_images_dir, source_labels_dir, output_images_dir, output_labels_dir, tile_size=tile_size, overlap=overlap)
    print("Tiling de todas las imágenes completado.")

    # Mostrar el conteo total de archivos generados en las carpetas de destino
    total_images = len(os.listdir(output_images_dir))
    total_labels = len(os.listdir(output_labels_dir))
    print(f"\nTotal de archivos generados en la carpeta de imágenes: {total_images}")
    print(f"Total de archivos generados en la carpeta de etiquetas: {total_labels}")


Procesada DSC05010_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.d7ca74e5bb5a68e04af68f8ee9f4d48e.jpg: 96 tiles generados.
En total, 96 tiles de imágenes generados.
En total, 96 tiles de etiquetas generados.
Procesada DSC05025_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.1f18312d52c2cd09fd891f8d6be14d4d.jpg: 96 tiles generados.
En total, 96 tiles de imágenes generados.
En total, 96 tiles de etiquetas generados.
Procesada DSC05026_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.9d10fb0e6c13ca76f30dfc04dae67792.jpg: 96 tiles generados.
En total, 96 tiles de imágenes generados.
En total, 96 tiles de etiquetas generados.
Procesada DSC05027_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.836550844aa218869cdefbf934148977.jpg: 96 tiles generados.
En total, 96 tiles de imágenes generados.
En total, 96 tiles de etiquetas generados.
Procesada DSC05032_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.ff2ab7f3ce9b926892f04866cf2a2340.jpg: 96 tiles generados.
En total, 96 tiles de imágenes generados.
En total, 96 tiles de etiquetas g

#5. OVERSAMPLING:

Aplicamos oversampling a aquellas fotos con etiquetas .

Aquí elegimos cuantas copias de nuestra foto orginal queremos producir.

El objetivo es aumentar el número de fotos positivas solo para poder tener más que enseñar.

---

In [ ]:
import os
import shutil

# Rutas de las carpetas originales
orig_images_dir = "/content/drive/MyDrive/Fotos_elegidas/train/images.rar"
orig_labels_dir = "/content/drive/MyDrive/Fotos_elegidas/train/labels"

# Carpeta de destino para el dataset balanceado
balanced_dir = "/content/balanced_train"
balanced_images_dir = os.path.join(balanced_dir, "images")
balanced_labels_dir = os.path.join(balanced_dir, "labels")

os.makedirs(balanced_images_dir, exist_ok=True)
os.makedirs(balanced_labels_dir, exist_ok=True)

################################################################
################################################################
# Factor de duplicación para imágenes positivas
dup_factor = 3                                                      ########cuantas veces las quiero multiplicar

#################################################################

# Recorremos todas las imágenes del dataset original
for filename in os.listdir(orig_images_dir):
    if not filename.lower().endswith(('.jpg','.jpeg','.png')):
        continue
    # Ruta de la imagen y de la etiqueta correspondiente
    orig_img_path = os.path.join(orig_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    orig_label_path = os.path.join(orig_labels_dir, label_filename)

    # Determinar si la imagen es positiva (tiene anotaciones) o negativa
    is_positive = False
    if os.path.exists(orig_label_path):
        with open(orig_label_path, 'r') as f:
            content = f.read().strip()
        if content:  # Si hay algo, es positiva
            is_positive = True

    # Copiar la imagen y el archivo de etiqueta al nuevo directorio
    # Si es positiva, duplicamos según el factor
    copies = dup_factor if is_positive else 1
    for i in range(copies):
        # Para evitar sobreescritura, agregamos un sufijo en el nombre de la imagen duplicada
        new_filename = os.path.splitext(filename)[0]
        if copies > 1:
            new_filename += f"_{i+1}"
        new_filename += os.path.splitext(filename)[1]
        new_img_path = os.path.join(balanced_images_dir, new_filename)
        shutil.copy2(orig_img_path, new_img_path)

        # También copiamos la etiqueta (si existe), ajustando el nombre
        if os.path.exists(orig_label_path):
            new_label_filename = os.path.splitext(label_filename)[0]
            if copies > 1:
                new_label_filename += f"_{i+1}"
            new_label_filename += ".txt"
            new_label_path = os.path.join(balanced_labels_dir, new_label_filename)
            shutil.copy2(orig_label_path, new_label_path)

print("Dataset balanceado creado en:", balanced_dir)


# 5. Data Augmentation

# se produce data augmentation solo en aquellas fotos con las clases indicadas:

Si quieres aplicar transformaciones a varias clases (por ejemplo, “0” y “1”), añádelas al set TARGET_CLASSES = {"0", "1"}.

Si deseas aplicar a todas las clases sin excepción, basta con eliminar el if not should_augment: y aplicar augmentation siempre, o bien poner TARGET_CLASSES = set(classes_que_tengas).

Ajusta la cantidad de variantes (num_variants = random.randint(3, 8)) a tu gusto.

Ten en cuenta que mucha data augmentation puede producir un dataset muy grande; controla tu espacio en disco y tus necesidades de entrenamiento.



Se produce un numero random de imagenes transformadas que incluyes transformaciones aleatorias

puedes controlar numero transformaciones, y numero de imagenes generadas

todo mas la original y label

In [15]:
import os
import cv2
import albumentations as A
import random
import shutil

def read_yolo_labels(label_path):
    """
    Lee un archivo de etiquetas YOLO y devuelve:
    - bboxes: [x_center, y_center, width, height] (normalizados)
    - classes: lista de clases (en formato string) correspondiente a cada bbox
    """
    bboxes = []
    classes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    classes.append(parts[0])           # la clase (ej: '0')
                    bbox = list(map(float, parts[1:])) # x_center, y_center, w, h
                    bboxes.append(bbox)
    return bboxes, classes

####################################################################################################################################
####################################################################################################################################

# === 1) ESPECIFICA AQUÍ LAS CLASES A LAS QUE QUIERES APLICAR DATA AUG ===
# Por ejemplo, si quieres data augmentation sólo para la clase '0' y '2':
TARGET_CLASSES = {"0","1","2"}
# (Si quieres aplicarlo a todas las clases, pon: TARGET_CLASSES = set(obj_list) o algo similar.)

####################################################################################################################################
####################################################################################################################################

# === 2) Definir la pipeline de Albumentations ===
transform_pipeline = A.Compose([
    A.SomeOf([
        A.HorizontalFlip(p=1.0),   # voltear horizontalmente
        A.Rotate(limit=(-5, 5), p=1.0),
        A.Affine(shear=(-5, 5), p=1.0),
        A.RandomBrightnessContrast(brightness_limit=(-0.05, 0.05),
                                   contrast_limit=(-0.05, 0.05),
                                   p=1.0),
        A.Blur(blur_limit=5, p=1.0),
        A.CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8)),  # Mejorar el contraste local
        A.ToGray(p=0.01, num_output_channels=3, method='weighted_average')  # Convertir a gris con 3 canales
        #A.ColorJitter(brightness=0.05,
                      #contrast=0.05,
                      #saturation=0.05,
                      #hue=0.2,
                      #p=1.0),
    ],



####################################################################################################################################
####################################################################################################################################

    n=(3),         # elige 3 transformaciones de la lista
    replace=False), # permite repeticiones
    A.Resize(640, 640)


####################################################################################################################################
####################################################################################################################################
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

# === 3) Rutas de entrada (imágenes y TXT) ===
input_images_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/train/tiles/images"
input_labels_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/train/tiles/labels"

# === 4) Rutas de salida para imágenes “positivas” y “negativas” ===
output_images_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/train/images"
output_labels_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/train/labels"
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

output_neg_images_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/train/tiles/negative/images"
output_neg_labels_dir = "/content/drive/MyDrive/AWS.v39i.seals+holes/train/tiles/negatives/labels"
os.makedirs(output_neg_images_dir, exist_ok=True)
os.makedirs(output_neg_labels_dir, exist_ok=True)




# === 5) Iterar sobre cada imagen en el directorio de entrada ===
for filename in os.listdir(input_images_dir):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(input_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(input_labels_dir, label_filename)

    # Leer bboxes y clases
    bboxes, classes = read_yolo_labels(label_path)

    # Si no hay anotaciones => imagen negativa
    if len(bboxes) == 0:
        shutil.copy2(img_path, os.path.join(output_neg_images_dir, filename))
        # crear un txt vacío
        open(os.path.join(output_neg_labels_dir, label_filename), 'w').close()
        print(f"{filename}: Sin anotaciones => copiada a negatives.")
        continue

    # Caso imagen con bboxes => imagen “positiva”
    # Copiamos la original (imagen + txt) a la carpeta positives
    shutil.copy2(img_path, os.path.join(output_images_dir, filename))
    shutil.copy2(label_path, os.path.join(output_labels_dir, label_filename))

    # === 6) Ver si se debe aplicar Data Aug. según las clases detectadas ===
    # Comprobamos si AL MENOS UNA clase de la imagen está en TARGET_CLASSES
    should_augment = any(cls_id in TARGET_CLASSES for cls_id in classes)

    if not should_augment:
        # Si no coincide ninguna clase con las “target”, NO generamos variantes
        print(f"{filename}: NO coincide con las clases {TARGET_CLASSES}, no se aplica augmentation.")
        continue

####################################################################################################################################
####################################################################################################################################

    # === 7) Generar un número aleatorio de imágenes aumentadas (p.ej. 3..8) ===
    num_variants = random.randint(3, 8)


####################################################################################################################################
####################################################################################################################################

    for i in range(num_variants):
        image_bgr = cv2.imread(img_path)
        if image_bgr is None:
            print(f"No se pudo leer {filename}, se omite.")
            continue

        # Convertir BGR -> RGB
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

        # Aplicar la pipeline
        try:
            result = transform_pipeline(
                image=image_rgb,
                bboxes=bboxes,
                labels=classes
            )
        except Exception as e:
            print(f"Error en pipeline con {filename}: {e}")
            continue

        aug_img_rgb = result["image"]
        aug_bboxes = result["bboxes"]
        aug_labels = result["labels"]  # en este caso es lo mismo que classes

        # Regresar a BGR
        aug_img_bgr = cv2.cvtColor(aug_img_rgb, cv2.COLOR_RGB2BGR)

        # Crear nombres de archivo
        new_img_name = f"{os.path.splitext(filename)[0]}_aug{i}.jpg"
        new_label_name = f"{os.path.splitext(filename)[0]}_aug{i}.txt"

        # Guardar la imagen
        cv2.imwrite(os.path.join(output_images_dir, new_img_name), aug_img_bgr)

        # Guardar anotaciones YOLO
        with open(os.path.join(output_labels_dir, new_label_name), 'w') as f:
            for cls_id, bbox in zip(aug_labels, aug_bboxes):
                # bbox es [x_center, y_center, w, h]
                f.write(f"{cls_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

    print(f"{filename}: Generadas {num_variants} variantes (clases={classes}, target={TARGET_CLASSES}).")

print("Data augmentation completo.")


Se han truncado las últimas 5000 líneas del flujo de salida.
DSC08283_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.3d03241dc5ef6c2d7aba8f8d67e3a4e2_tile_5.jpg: Sin anotaciones => copiada a negatives.
DSC08283_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.3d03241dc5ef6c2d7aba8f8d67e3a4e2_tile_6.jpg: Sin anotaciones => copiada a negatives.
DSC08283_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.3d03241dc5ef6c2d7aba8f8d67e3a4e2_tile_7.jpg: Sin anotaciones => copiada a negatives.
DSC08283_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.3d03241dc5ef6c2d7aba8f8d67e3a4e2_tile_8.jpg: Sin anotaciones => copiada a negatives.
DSC08283_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.3d03241dc5ef6c2d7aba8f8d67e3a4e2_tile_9.jpg: Sin anotaciones => copiada a negatives.
DSC08283_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.3d03241dc5ef6c2d7aba8f8d67e3a4e2_tile_10.jpg: Sin anotaciones => copiada a negatives.
DSC08283_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.3d03241dc5ef6c2d7aba8f8d67e3a4e2_tile_11.jpg: Sin anotaciones => copiada a negatives.

# sin clases

In [9]:
import os
import cv2
import albumentations as A
import random
import shutil

def read_yolo_labels(label_path):
    """
    Lee un archivo de etiquetas YOLO y devuelve:
    - bboxes: [x_center, y_center, width, height] (normalizados)
    - classes: Lista de clases correspondiente a cada bbox
    """
    bboxes = []
    classes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    classes.append(parts[0])  # Clase de la anotación (id de clase)
                    bbox = list(map(float, parts[1:]))  # Coordenadas del bounding box
                    bboxes.append(bbox)
    return bboxes, classes

# 1) Definir una pipeline que escoja aleatoriamente entre 1 y 6 transformaciones
#    con replace=True para permitir tupla en 'n'
transform_pipeline = A.Compose([
    A.SomeOf([
        A.HorizontalFlip(p=1.0),  # Voltear horizontalmente
        A.Rotate(limit=(-5, 5), p=1.0),  # Rotación aleatoria entre -5° y +5°
        A.Affine(shear=(-5, 5), p=1.0),  # Cizallamiento aleatorio entre -5° y +5°
        A.RandomBrightnessContrast(brightness_limit=(-0.05, 0.05), contrast_limit=(-0.05, 0.05), p=1.0),
        A.Blur(blur_limit=5, p=1.0),  # Aplicar desenfoque
        A.CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8)),  # Mejorar el contraste local
        A.ToGray(p=0.01, num_output_channels=3, method='weighted_average'),  # Convertir a gris con 3 canales
     ],

####################################################################################################################################
####################################################################################################################################
    n=(4),        # elige un subconjunto aleatorio de 1 a 6 transformaciones
    replace=True),   # <-- IMPORTANTE: así evitamos la comparación n > len(self.transforms)
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))



####################################################################################################################################
####################################################################################################################################
# Directorios de entrada (donde están tus imágenes y anotaciones)
input_images_dir = "/content/drive/MyDrive/prueba_data_aug/images"
input_labels_dir = "/content/drive/MyDrive/prueba_data_aug/images/labels"

# Directorios de salida para imágenes positivas
output_images_dir = "/content/drive/MyDrive/prueba_data_aug/prueba/positives/images"
output_labels_dir = "/content/drive/MyDrive/prueba_data_aug/prueba/positives/labels"
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Directorios de salida para imágenes negativas
output_neg_images_dir = "/content/drive/MyDrive/prueba_data_aug/prueba/negatives/images"
output_neg_labels_dir = "/content/drive/MyDrive/prueba_data_aug/prueba/negatives/labels"
os.makedirs(output_neg_images_dir, exist_ok=True)
os.makedirs(output_neg_labels_dir, exist_ok=True)

# 2) Iterar sobre cada imagen
for filename in os.listdir(input_images_dir):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(input_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(input_labels_dir, label_filename)

    bboxes, classes = read_yolo_labels(label_path)

    # 2a) Si no hay anotaciones, se considera "negativa"
    if len(bboxes) == 0:
        # Copiamos la imagen original y un TXT vacío
        shutil.copy2(img_path, os.path.join(output_neg_images_dir, filename))
        open(os.path.join(output_neg_labels_dir, label_filename), 'w').close()
        print(f"{filename}: No tiene anotaciones. Copiada en negativos.")
        continue

    # 2b) Si la imagen tiene anotaciones, la copiamos como "positiva"
    shutil.copy2(img_path, os.path.join(output_images_dir, filename))
    shutil.copy2(label_path, os.path.join(output_labels_dir, label_filename))
####################################################################################################################################
####################################################################################################################################

    # 3) Generar un número aleatorio de imágenes aumentadas (entre 1 y 10)

    num_variants = random.randint(4, 6)

####################################################################################################################################
####################################################################################################################################

    # 4) Crear las variantes
    for i in range(num_variants):
        # Leer la imagen original en BGR
        image_bgr = cv2.imread(img_path)
        if image_bgr is None:
            print(f"No se pudo leer {filename}, se omite.")
            continue

        # Convertir a RGB para Albumentations
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

        # Aplicar la pipeline
        try:
            result = transform_pipeline(
                image=image_rgb,
                bboxes=bboxes,
                labels=classes
            )
        except Exception as e:
            print(f"Error al aplicar pipeline en {filename}: {e}")
            continue

        aug_img_rgb = result["image"]
        aug_bboxes = result["bboxes"]

        # Convertir de vuelta a BGR para guardar
        aug_img_bgr = cv2.cvtColor(aug_img_rgb, cv2.COLOR_RGB2BGR)

        # Nuevo nombre de archivo
        new_img_name = f"{os.path.splitext(filename)[0]}_aug{i}.jpg"
        new_label_name = f"{os.path.splitext(filename)[0]}_aug{i}.txt"

        # Guardar la imagen aumentada
        cv2.imwrite(os.path.join(output_images_dir, new_img_name), aug_img_bgr)

        # Guardar las nuevas anotaciones en formato YOLO
        with open(os.path.join(output_labels_dir, new_label_name), 'w') as f:
            for cls, bbox in zip(classes, aug_bboxes):
                f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

        print(f"Procesada {filename}: generadas {num_variants} variantes.")

print("Data augmentation completo.")


DSC05010_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.d7ca74e5bb5a68e04af68f8ee9f4d48e_tile_7.jpg: No tiene anotaciones. Copiada en negativos.
DSC05010_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.d7ca74e5bb5a68e04af68f8ee9f4d48e_tile_8.jpg: No tiene anotaciones. Copiada en negativos.
DSC05025_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.1f18312d52c2cd09fd891f8d6be14d4d_tile_32.jpg: No tiene anotaciones. Copiada en negativos.
DSC05025_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.1f18312d52c2cd09fd891f8d6be14d4d_tile_44.jpg: No tiene anotaciones. Copiada en negativos.
DSC05025_JPG-X-Amz-Algorithm-AWS4-HMAC-SHA256.rf.1f18312d52c2cd09fd891f8d6be14d4d_tile_45.jpg: No tiene anotaciones. Copiada en negativos.
Data augmentation completo.


# 6.3 Calcular números archivos carpeta para hacer proporciones imágenes positivas y negativas

In [13]:
import os

# Rutas de las dos carpetas
folder1 = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/valid/images"
folder2 = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/valid/labels"

def count_files_in_folder(folder_path):
    """
    Cuenta el número de archivos (no directorios) en la carpeta dada.
    """
    count = 0
    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isfile(full_path):
            count += 1
    return count

# Contar archivos en cada carpeta
count1 = count_files_in_folder(folder1)
count2 = count_files_in_folder(folder2)

# Mostrar resultados
print(f"Carpeta 1: {count1} archivos")
print(f"Carpeta 2: {count2} archivos")
print(f"Total de archivos en cada carpeta: Imágenes {count1} + Labels: {count2}")

Carpeta 1: 0 archivos
Carpeta 2: 576 archivos
Total de archivos en cada carpeta: Imágenes 0 + Labels: 576


producir archivos posit y negat para validacion tras tiling solo

In [ ]:
import os
import shutil

def move_files_by_annotations(labels_folder, images_folder, output_folder):
    """
    Mueve archivos de imágenes y etiquetas YOLO a subcarpetas según si tienen anotaciones.

    Args:
        labels_folder (str): Ruta a la carpeta que contiene los archivos de etiquetas .txt.
        images_folder (str): Ruta a la carpeta que contiene las imágenes correspondientes.
        output_folder (str): Ruta a la carpeta donde se crearán las subcarpetas de salida.
    """
    # Crear subcarpetas para archivos positivos y negativos
    positive_images_folder = os.path.join(output_folder, "positive", "images")
    positive_labels_folder = os.path.join(output_folder, "positive", "labels")
    negative_images_folder = os.path.join(output_folder, "negative", "images")
    negative_labels_folder = os.path.join(output_folder, "negative", "labels")

    os.makedirs(positive_images_folder, exist_ok=True)
    os.makedirs(positive_labels_folder, exist_ok=True)
    os.makedirs(negative_images_folder, exist_ok=True)
    os.makedirs(negative_labels_folder, exist_ok=True)

    txt_files = [f for f in os.listdir(labels_folder) if f.lower().endswith('.txt')]

    for txt_file in txt_files:
        txt_path = os.path.join(labels_folder, txt_file)
        image_file = txt_file.replace('.txt', '.jpg')  # Asumiendo que las imágenes son .jpg
        image_path = os.path.join(images_folder, image_file)

        # Verificar si el archivo .txt tiene anotaciones
        with open(txt_path, 'r') as f:
            annotations = f.readlines()

        if annotations:  # Si el archivo tiene anotaciones (no está vacío)
            shutil.move(txt_path, os.path.join(positive_labels_folder, txt_file))
            if os.path.exists(image_path):
                shutil.move(image_path, os.path.join(positive_images_folder, image_file))
            else:
                print(f"Advertencia: No se encontró la imagen {image_file} correspondiente a {txt_file}.")
        else:  # Si el archivo no tiene anotaciones
            shutil.move(txt_path, os.path.join(negative_labels_folder, txt_file))
            if os.path.exists(image_path):
                shutil.move(image_path, os.path.join(negative_images_folder, image_file))
            else:
                print(f"Advertencia: No se encontró la imagen {image_file} correspondiente a {txt_file}.")

    # Contar los archivos generados en las carpetas
    positive_images_count = len([f for f in os.listdir(positive_images_folder) if os.path.isfile(os.path.join(positive_images_folder, f))])
    positive_labels_count = len([f for f in os.listdir(positive_labels_folder) if os.path.isfile(os.path.join(positive_labels_folder, f))])
    negative_images_count = len([f for f in os.listdir(negative_images_folder) if os.path.isfile(os.path.join(negative_images_folder, f))])
    negative_labels_count = len([f for f in os.listdir(negative_labels_folder) if os.path.isfile(os.path.join(negative_labels_folder, f))])

    # Imprimir el número de archivos generados
    print(f"Total de archivos generados en 'positive/images': {positive_images_count}")
    print(f"Total de archivos generados en 'positive/labels': {positive_labels_count}")
    print(f"Total de archivos generados en 'negative/images': {negative_images_count}")
    print(f"Total de archivos generados en 'negative/labels': {negative_labels_count}")

# Ejemplo de uso
if __name__ == "__main__":
    labels_folder = "/content/drive/MyDrive/AWS.v39i.seals+holes/valid/tiles/labels"  # Ajusta esta ruta
    images_folder = "/content/drive/MyDrive/AWS.v39i.seals+holes/valid/tiles/images"  # Ajusta esta ruta
    output_folder = "/content/drive/MyDrive/AWS.v39i.seals+holes/valid/tiles/sorted_data"  # Ajusta esta ruta

    move_files_by_annotations(labels_folder, images_folder, output_folder)
    print("Proceso completado.")


mandar archivos de una carpeta a otra, un numero que tu eliges

In [ ]:
import os
import shutil

def copiar_archivos_en_orden(carpeta_origen, carpeta_destino, num_archivos):
    """
    Copia los primeros num_archivos de la carpeta_origen a la carpeta_destino
    en orden alfabético. Si la carpeta destino no existe, se crea.

    Args:
        carpeta_origen (str): La ruta de la carpeta de origen.
        carpeta_destino (str): La ruta de la carpeta de destino.
        num_archivos (int): El número de archivos a copiar.
    """
    # Crear carpeta de destino si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
        print(f"Carpeta de destino '{carpeta_destino}' creada.")

    archivos = [f for f in os.listdir(carpeta_origen) if os.path.isfile(os.path.join(carpeta_origen, f))]
    archivos_ordenados = sorted(archivos)  # Orden alfabético

    if num_archivos > len(archivos_ordenados):
        print(f"Advertencia: Solo hay {len(archivos_ordenados)} archivos en la carpeta de origen.")
        num_archivos = len(archivos_ordenados)

    for i in range(num_archivos):
        archivo_origen = os.path.join(carpeta_origen, archivos_ordenados[i])
        archivo_destino = os.path.join(carpeta_destino, archivos_ordenados[i])
        shutil.copy2(archivo_origen, archivo_destino)
        print(f"Copiado: {archivos_ordenados[i]}")

# === Ejemplo de uso ===
carpeta_origen = "/content/drive/MyDrive/AWS.v39i.seals+holes/valid/tiles/sorted_data/negative/images"
carpeta_destino = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/valid/images"
num_archivos = 480

copiar_archivos_en_orden(carpeta_origen, carpeta_destino, num_archivos)


In [ ]:
import os

def eliminar_imagenes_sin_label(carpeta_imagenes, carpeta_labels):
    """
    Elimina imágenes .jpg/.jpeg/.png si no existe un archivo .txt con el mismo nombre en carpeta_labels.
    """
    imagenes = [f for f in os.listdir(carpeta_imagenes) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    eliminadas = 0

    for imagen in imagenes:
        nombre_base = os.path.splitext(imagen)[0]
        label_path = os.path.join(carpeta_labels, nombre_base + ".txt")

        if not os.path.exists(label_path):
            ruta_imagen = os.path.join(carpeta_imagenes, imagen)
            os.remove(ruta_imagen)
            eliminadas += 1
            print(f"🗑️ Eliminada: {imagen}")

    print(f"\n✅ Total de imágenes eliminadas por no tener archivo .txt: {eliminadas}")

# Ejemplo de uso
carpeta_imagenes = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/train/images"  # Cambia por tu carpeta de imágenes
carpeta_labels = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/train/labels"    # Cambia por tu carpeta de labels

eliminar_imagenes_sin_label(carpeta_imagenes, carpeta_labels)
